In [1]:
import pandas as pd
import random

In [2]:
df = pd.read_csv('../data/monday.csv', sep=';', parse_dates=True, index_col=[0])
df.head(3)

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:03:00,2,dairy
2019-09-02 07:04:00,3,dairy
